In [1]:
import numpy as np
import cv2

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

from srgan import SRGAN
from generator import ImageDataGenerator

Using TensorFlow backend.


In [2]:
# !TF_ENABLE_GPU_GARBAGE_COLLECTION=false

In [3]:
# from tensorflow.python.client import device_lib
# device_lib.list_local_devices()

In [4]:
input_dir = 'data/train/low/'
tgt_dir = 'data/train/high/'

input_dir_val = 'data/val/low/'
tgt_dir_val = 'data/val/high/'


test_file = 'UBMk30rjy0o_17675_1.jpg'
test_low_path =  input_dir_val + test_file
test_high_path = tgt_dir_val + test_file
test_out_path = 'data/out/' + test_file
test_test_path = 'data/test/' + test_file

vgg_path = 'vgg16_notop.hdf5'

input_shape = (135, 240, 3)
tgt_shape = (540, 960, 3)

batch_size = 2

In [5]:
srgan = SRGAN(vgg_path, batch_size, input_shape, tgt_shape)
gen = ImageDataGenerator(input_dir, tgt_dir, batch_size)
gen_valid = ImageDataGenerator(input_dir_val, tgt_dir_val, batch_size)

/home/keras/srgan.py:141: UserWarning: Update your `Network` call to the Keras 2 API: `Network(name="generator", inputs=Tensor("in..., outputs=Tensor("co...)`
  self.shared_generator = Network(input=layers[0], output=layers[-1], name='generator')
/home/keras/srgan.py:178: UserWarning: Update your `Network` call to the Keras 2 API: `Network(name="discriminator", inputs=Tensor("in..., outputs=Tensor("de...)`
  self.shared_discriminator = Network(input=layers[0], output=layers[-1], name='discriminator')
/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [6]:
# test_img_low = ((cv2.imread(test_low_path) - 127.5) / 127.5)[np.newaxis, :, :, :]
# test_img_high = ((cv2.imread(test_high_path)- 127.5) / 127.5)[np.newaxis, :, :, :]

# test_img_sr = srgan.generator.predict(test_img_low)
# test_img_sr_int = ((test_img_sr * 127.5) + 127.5).astype('u1')[0]
# cv2.imwrite(test_test_path, test_img_sr_int)

In [ ]:
for e in range(10000):
    X_low, X_high = gen.flow_from_directory().__next__()
    srgan.train(X_low, X_high)
    if e % 100 == 0:
        print('epoch', e)
        srgan.valid(X_low, X_high)
        
        test_img_low = ((cv2.imread(test_low_path) - 127.5) / 127.5)[np.newaxis, :, :, :]
        test_img_high = ((cv2.imread(test_high_path)- 127.5) / 127.5)[np.newaxis, :, :, :]

        test_img_sr = srgan.generator.predict(test_img_low)
        test_img_sr_int = ((test_img_sr * 127.5) + 127.5).astype('u1')[0]
        test_out_path = 'data/out/'  + test_file.split('.')[0] + f'_test_{e}.jpg'
        cv2.imwrite(test_out_path, test_img_sr_int)

epoch 0
d_loss_real:1.1174440383911133, d_loss_fake:0.39869338274002075, vgg_loss:92.28553771972656


In [ ]:
# test_out_path = 'data/out/'  + test_file.split('.')[0] + f'_test_{e}.jpg'

# test_img_low = ((cv2.imread(test_low_path) - 127.5) / 127.5)[np.newaxis, :, :, :]
# test_img_high = ((cv2.imread(test_high_path)- 127.5) / 127.5)[np.newaxis, :, :, :]

# test_img_sr = srgan.generator.predict(test_img_low)
# test_img_sr_int = ((test_img_sr * 127.5) + 127.5).astype('u1')[0]
# cv2.imwrite(test_out_path, test_img_sr_int)